# TODO

Variables to explore:

- [x] PN17
- [x] PN25
- [x] PN34
- [x] PN35
- [ ] PN36
- [ ] PN38
- [ ] PN9

## Set up

In [ ]:
%load_ext rich
%load_ext autoreload
%autoreload 2

The rich extension is already loaded. To reload it, use:
  %reload_ext rich
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from pathlib import Path
import polars as pl
import pandas as pd
import numpy as np
from rich import print as rprint

from pain.read import *
from pain.explore import *

In [ ]:
data_dir = Path("../data/raw")

In [ ]:
datasets = [
    Dataset("G214_PQ.sav", data_dir, "G214_PQ_", ["ID", "G214_PQ_PN17", "G214_PQ_PN25", "G214_PQ_PN34", "G214_PQ_PN35", "G214_PQ_PN36"]),
    Dataset("G214_SQ.sav", data_dir, "G214_SQ_", ["ID", "G214_SQ_PN17", "G214_SQ_PN25", "G214_SQ_PN34", "G214_SQ_PN35", "G214_SQ_PN36"]),
    Dataset("G217_PQ.sav", data_dir, "G217_PQ_", ["ID", "G217_PQ_PN17", "G217_PQ_PN25", "G217_PQ_PN34", "G217_PQ_PN35", "G217_PQ_PN36", "G217_PQ_PN38", "G217_PQ_PN9"]),
    Dataset("G217_SQ.sav", data_dir, "G217_SQ_", ["ID", "G217_SQ_PN17", "G217_SQ_PN25", "G217_SQ_PN34", "G217_SQ_PN35", "G217_SQ_PN36", "G217_SQ_PN38", "G217_SQ_PN9"])
]

prefixes = ["G214_PQ_", "G214_SQ_", "G217_PQ_", "G217_SQ_"]

In [ ]:
dataframes, metadata = read_and_filter_data(datasets)
df = combine_dataframes(dataframes)
meta = merge_dictionaries(metadata)

## PN17

In [ ]:
var = "PN17"

### Data

The existing data options for PN17 are all the same:
- 0: No
- 1: Yes
- 9: Missing

Changes to make:
- Convert 9 to -99 in all cases

In [ ]:
unique_values(df, var)


{
    'G214_PQ_PN17': (None, 0.0, 1.0, 9.0),
    'G214_SQ_PN17': (None, 0.0, 1.0, 9.0),
    'G217_PQ_PN17': (None, 0.0, 1.0, 9.0),
    'G217_SQ_PN17': (None, 0.0, 1.0, 9.0)
}

### Metadata

The metadata is generally identical, with a minor discrepancy in the label for G217_SQ.
There are differences in value labels, but these values do not exist in the data, so they can be harmonised without changes to the raw data.

In [ ]:
m = filter_metadata(var, df, meta)
pd.DataFrame(m).T

,G214_PQ_PN17,G214_SQ_PN17,G217_PQ_PN17,G217_SQ_PN17
Label,Ever had back pain,Ever had back pain,Ever had back pain,Ever had back pain?
Field Type,Numeric,Numeric,Numeric,Numeric
Field Width,8,8,8,8
Decimals,0,0,0,0
Variable Type,scale,scale,scale,scale
Field Values,"{0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable',...","{0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable',...","{0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in inco...","{0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}"


## PN25

In [ ]:
var = "PN25"

### Data

Some of the existing options are the same across all datasets:
- 0: No
- 1: Yes
- 9: Missing

For G214_PQ and G214_SQ
- 8: N/A

For G217_PQ:
- 7: Involved in incorrect skip - not answered

Values of 7 should be considered missing, as discussed with Alex D'Vauz.

Changes:
- Convert 8 to -88
- Convert 9 to -99
- Convert 7 to -99

In [ ]:
unique_vals = unique_values(df, var)
rprint(unique_vals)

{
    'G214_PQ_PN25': (None, 0.0, 1.0, 8.0, 9.0),
    'G214_SQ_PN25': (None, 0.0, 1.0, 8.0, 9.0),
    'G217_PQ_PN25': (None, 0.0, 1.0, 7.0, 9.0),
    'G217_SQ_PN25': (None, 0.0, 1.0, 9.0)
}

In [ ]:
# Note the unique values for PN25 across all datasets combined
value_options = set()
for values in unique_vals.values(): 
    value_options.update(values)
value_options.discard(None)
value_options

{0.0, 1.0, 7.0, 8.0, 9.0}

#### Define properties to test, validate, and explore

- When PN17 is 0 (No), PN25 should be 8 (N/A)
- When PN17 is 1 (Yes), PN25 should be 0, 1, or 9
- When PN17 is 9 (Missing), PN25 should be 9

In [ ]:
test = df.clone().collect()

In [ ]:
for value in (0, 1, 9):
    rprint(f"When PN17 == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}PN17") == value)
            .select(f"{p}{var}")
            .unique()
            .to_dict(as_series=False)
        )

When PN17 == 0

{'G214_PQ_PN25': [8.0]}

{'G214_SQ_PN25': [8.0]}

{'G217_PQ_PN25': [9.0, 1.0, 7.0, 0.0]}

{'G217_SQ_PN25': [0.0, 1.0, 9.0]}

...

When PN17 == 9

{'G214_PQ_PN25': [9.0]}

{'G214_SQ_PN25': [9.0]}

{'G217_PQ_PN25': [9.0]}

{'G217_SQ_PN25': [9.0]}

Properties 2 and 3 were as expected.
Property 1 did not pass, as there were differences in the Y17 follow-ups.
On further investigation, it appears there were two additional questions, related to neck, and shoulder pain, which changed the following logic of the questions.
For Y14, is the participant responded "No" to PN17, all following questions were skipped.
For Y17, they may have responded "No" to PN17, but if they answered "Yes" to having either neck or shoulder pain, they still answered the subsequent questions; hence, values of 0, 1 and 9 (and 7 due to incorrect skips).

In [ ]:
for value in value_options:
    rprint(f"When {var} == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}{var}") == value)
            .select(f"{p}PN17")
            .unique()
            .to_dict(as_series=False)
        )

When PN25 == 0.0

{'G214_PQ_PN17': [1.0]}

{'G214_SQ_PN17': [1.0]}

{'G217_PQ_PN17': [1.0, 0.0]}

{'G217_SQ_PN17': [0.0, 1.0]}

...

When PN25 == 9.0

{'G214_PQ_PN17': [9.0]}

{'G214_SQ_PN17': [9.0, 1.0]}

{'G217_PQ_PN17': [1.0, 0.0, 9.0]}

{'G217_SQ_PN17': [0.0, 9.0]}

No additional, unexpected values were found.

### Metadata

Clear discrepancies in labels and field values.

Changes:
- Harmonised label to "Sought professional advice/treatment"
- Updated field values to reflect changes in data

In [ ]:
m = filter_metadata(var, df, meta)

In [ ]:
rprint(m)

defaultdict(<class 'dict'>, {
    'Label': {
        'G214_PQ_PN25': 'Seek pro advice for back pain',
        'G214_SQ_PN25': 'Seek pro advice for back pain',
        'G217_PQ_PN25': 'Ever sought health professional advice/treatment for back pain',
        'G217_SQ_PN25': 'Seek treatment for back pain?'
    },
    'Field Type': {
        'G214_PQ_PN25': 'Numeric',
        'G214_SQ_PN25': 'Numeric',
        'G217_PQ_PN25': 'Numeric',
        'G217_SQ_PN25': 'Numeric'
    },
    'Field Width': {'G214_PQ_PN25': 8, 'G214_SQ_PN25': 8, 'G217_PQ_PN25': 8, 'G217_SQ_PN25': 8},
    'Decimals': {'G214_PQ_PN25': '0', 'G214_SQ_PN25': '0', 'G217_PQ_PN25': '0', 'G217_SQ_PN25': '0'},
    'Variable Type': {
        'G214_PQ_PN25': 'scale',
        'G214_SQ_PN25': 'scale',
        'G217_PQ_PN25': 'scale',
        'G217_SQ_PN25': 'scale'
    },
    'Field Values': {
        'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G217_PQ_PN25': {
            0.0: 'No',
            1.0: 'Yes',
            7.0: 'Involved in incorrect skip - not answered',
            9.0: 'Not stated'
        },
        'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
    }
})

In [ ]:
rprint(m["Field Values"])

{
    'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G217_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in incorrect skip - not answered', 9.0: 'Not stated'},
    'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
}

In [ ]:
PN25 = Metadata(
    label= "Sought professional advice/treatment",
    field_values = {-88: "N/A", -99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

## PN34

In [ ]:
var = "PN34"

### Data

Some of the existing options are the same across all datasets:
- 0: No
- 1: Yes
- 9: Missing

For G214_PQ and G214_SQ
- 8: N/A

For G217_PQ:
- 7: Involved in incorrect skip - not answered

Values of 7 should be considered missing, as discussed with Alex D'Vauz.

Changes:
- Convert 8 to -88
- Convert 9 to -99
- Convert 7 to -99

In [ ]:
unique_vals = unique_values(df, var)
rprint(unique_vals)

{
    'G214_PQ_PN34': (None, 0.0, 1.0, 8.0, 9.0),
    'G214_SQ_PN34': (None, 0.0, 1.0, 8.0, 9.0),
    'G217_PQ_PN34': (None, 0.0, 1.0, 7.0, 9.0),
    'G217_SQ_PN34': (None, 0.0, 1.0, 9.0)
}

In [ ]:
# Note the unique values for PN34 across all datasets combined
value_options = set()
for values in unique_vals.values(): 
    value_options.update(values)
value_options.discard(None)
value_options

{0.0, 1.0, 7.0, 8.0, 9.0}

#### Define properties to test, validate, and explore

- When PN17 is 0 (No), PN34 should be 8 (N/A)
- When PN17 is 1 (Yes), PN34 should be 0, 1, or 9
- When PN17 is 9 (Missing), PN34 should be 9

In [ ]:
test = df.clone().collect()

In [ ]:
for value in (0, 1, 9):
    rprint(f"When PN17 == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}PN17") == value)
            .select(f"{p}{var}")
            .unique()
            .to_dict(as_series=False)
        )

When PN17 == 0

{'G214_PQ_PN34': [8.0]}

{'G214_SQ_PN34': [8.0]}

{'G217_PQ_PN34': [0.0, 7.0, 1.0, 9.0]}

{'G217_SQ_PN34': [0.0, 1.0, 9.0]}

...

When PN17 == 9

{'G214_PQ_PN34': [9.0]}

{'G214_SQ_PN34': [9.0]}

{'G217_PQ_PN34': [9.0]}

{'G217_SQ_PN34': [9.0]}

As per PN25, properties 2 and 3 were as expected and property 1 did not pass due to differences in the Y17 follow-ups.

In [ ]:
for value in value_options:
    rprint(f"When {var} == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}{var}") == value)
            .select(f"{p}PN17")
            .unique()
            .to_dict(as_series=False)
        )

When PN34 == 0.0

{'G214_PQ_PN17': [1.0]}

{'G214_SQ_PN17': [1.0]}

{'G217_PQ_PN17': [0.0, 1.0]}

{'G217_SQ_PN17': [0.0, 1.0]}

...

When PN34 == 9.0

{'G214_PQ_PN17': [9.0]}

{'G214_SQ_PN17': [9.0, 1.0]}

{'G217_PQ_PN17': [9.0, 1.0, 0.0]}

{'G217_SQ_PN17': [9.0, 0.0]}

No additional, unexpected values were found.

### Metadata

Clear discrepancies in labels and field values.

Changes:
- Harmonised label to "Sought professional advice/treatment"
- Updated field values to reflect changes in data

In [ ]:
m = filter_metadata(var, df, meta)

In [ ]:
rprint(m)

defaultdict(<class 'dict'>, {
    'Label': {
        'G214_PQ_PN34': 'Takes meds to relieve back pain',
        'G214_SQ_PN34': 'Takes meds to relieve back pain',
        'G217_PQ_PN34': 'Ever taken medication to relieve back pain',
        'G217_SQ_PN34': 'Take any medication for back pain?'
    },
    'Field Type': {
        'G214_PQ_PN34': 'Numeric',
        'G214_SQ_PN34': 'Numeric',
        'G217_PQ_PN34': 'Numeric',
        'G217_SQ_PN34': 'Numeric'
    },
    'Field Width': {'G214_PQ_PN34': 8, 'G214_SQ_PN34': 8, 'G217_PQ_PN34': 8, 'G217_SQ_PN34': 8},
    'Decimals': {'G214_PQ_PN34': '0', 'G214_SQ_PN34': '0', 'G217_PQ_PN34': '0', 'G217_SQ_PN34': '0'},
    'Variable Type': {
        'G214_PQ_PN34': 'scale',
        'G214_SQ_PN34': 'scale',
        'G217_PQ_PN34': 'scale',
        'G217_SQ_PN34': 'scale'
    },
    'Field Values': {
        'G214_PQ_PN34': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G214_SQ_PN34': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G217_PQ_PN34': {
            0.0: 'No',
            1.0: 'Yes',
            7.0: 'Involved in incorrect skip - not answered',
            9.0: 'Not stated'
        },
        'G217_SQ_PN34': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
    }
})

In [ ]:
rprint(m["Field Values"])

{
    'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G217_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in incorrect skip - not answered', 9.0: 'Not stated'},
    'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
}

In [ ]:
PN34 = Metadata(
    label= "Took medication to relieve pain",
    field_values = {-88: "N/A", -99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

## PN35

In [ ]:
var = "PN35"

### Data

Some of the existing options are the same across all datasets:
- 0: No
- 1: Yes
- 9: Missing

For G214_PQ and G214_SQ
- 8: N/A

For G217_PQ:
- 7: Involved in incorrect skip - not answered

Values of 7 should be considered missing, as discussed with Alex D'Vauz.

Changes:
- Convert 8 to -88
- Convert 9 to -99
- Convert 7 to -99

In [ ]:
unique_vals = unique_values(df, var)
rprint(unique_vals)

{
    'G214_PQ_PN35': (None, 0.0, 1.0, 8.0, 9.0),
    'G214_SQ_PN35': (None, 0.0, 1.0, 8.0, 9.0),
    'G217_PQ_PN35': (None, 0.0, 1.0, 7.0, 9.0),
    'G217_SQ_PN35': (None, 0.0, 1.0, 9.0)
}

In [ ]:
# Note the unique values for PN35 across all datasets combined
value_options = set()
for values in unique_vals.values(): 
    value_options.update(values)
value_options.discard(None)
value_options

{0.0, 1.0, 7.0, 8.0, 9.0}

#### Define properties to test, validate, and explore

- When PN17 is 0 (No), PN35 should be 8 (N/A)
- When PN17 is 1 (Yes), PN35 should be 0, 1, or 9 [CORRECTION: as per discussion below, 8 is also valid for PN35]
- When PN17 is 9 (Missing), PN35 should be 9

In [ ]:
test = df.clone().collect()

In [ ]:
for value in (0, 1, 9):
    rprint(f"When PN17 == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}PN17") == value)
            .select(f"{p}{var}")
            .unique()
            .to_dict(as_series=False)
        )

When PN17 == 0

{'G214_PQ_PN35': [8.0]}

{'G214_SQ_PN35': [8.0]}

{'G217_PQ_PN35': [7.0, 1.0, 0.0, 9.0]}

{'G217_SQ_PN35': [1.0, 0.0, 9.0]}

...

When PN17 == 9

{'G214_PQ_PN35': [9.0]}

{'G214_SQ_PN35': [9.0]}

{'G217_PQ_PN35': [9.0]}

{'G217_SQ_PN35': [9.0]}

As per PN25, property 3 was as expected and property 1 did not pass due to differences in the Y17 follow-ups.
However, property 2 failed unexpectedly; there are values of 8 (N/A) when PN17 is 1 (Yes).

Upon further exploration, it was confirmed that in ALL cases, this is because the participant was not employed at the time of the questionnaire, and thus the response of 8 is valid.
This was done by checking values of `G214_PQ_YWRK_1` and `G214_PQ_YWRK_YN` when PN17 == 1 and PN35 == 8.

In [ ]:
p = "G214_PQ_"
(
    test
    .filter(
        pl.col(f"{p}PN17") == 1,
        pl.col(f"{p}PN35") == 8
    ).select(pl.col("^G214_PQ.*$"))
)

G214_PQ_PN17,G214_PQ_PN25,G214_PQ_PN34,G214_PQ_PN35,G214_PQ_PN36
f64,f64,f64,f64,f64
1.0,1.0,1.0,8.0,1.0
1.0,0.0,0.0,8.0,0.0
1.0,0.0,1.0,8.0,0.0
1.0,1.0,1.0,8.0,1.0
1.0,1.0,0.0,8.0,1.0
…,…,…,…,…
1.0,0.0,1.0,8.0,0.0
1.0,1.0,1.0,8.0,1.0
1.0,1.0,1.0,8.0,1.0


In [ ]:
for value in value_options:
    rprint(f"When {var} == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}{var}") == value)
            .select(f"{p}PN17")
            .unique()
            .to_dict(as_series=False)
        )

When PN35 == 0.0

{'G214_PQ_PN17': [1.0]}

{'G214_SQ_PN17': [1.0]}

{'G217_PQ_PN17': [0.0, 1.0]}

{'G217_SQ_PN17': [0.0, 1.0]}

...

When PN35 == 9.0

{'G214_PQ_PN17': [9.0]}

{'G214_SQ_PN17': [1.0, 9.0]}

{'G217_PQ_PN17': [1.0, 9.0, 0.0]}

{'G217_SQ_PN17': [9.0, 0.0]}

No additional, unexpected values were found.

### Metadata

Clear discrepancies in labels and field values.

Changes:
- Harmonised label to "Sought professional advice/treatment"
- Updated field values to reflect changes in data

In [ ]:
m = filter_metadata(var, df, meta)

In [ ]:
rprint(m)

defaultdict(<class 'dict'>, {
    'Label': {
        'G214_PQ_PN35': 'Miss work due to back pain',
        'G214_SQ_PN35': 'Miss work due to back pain',
        'G217_PQ_PN35': 'Back pain stopped you from going to work',
        'G217_SQ_PN35': 'Did back pain stop you going to work?'
    },
    'Field Type': {
        'G214_PQ_PN35': 'Numeric',
        'G214_SQ_PN35': 'Numeric',
        'G217_PQ_PN35': 'Numeric',
        'G217_SQ_PN35': 'Numeric'
    },
    'Field Width': {'G214_PQ_PN35': 8, 'G214_SQ_PN35': 8, 'G217_PQ_PN35': 8, 'G217_SQ_PN35': 8},
    'Decimals': {'G214_PQ_PN35': '0', 'G214_SQ_PN35': '0', 'G217_PQ_PN35': '0', 'G217_SQ_PN35': '0'},
    'Variable Type': {
        'G214_PQ_PN35': 'scale',
        'G214_SQ_PN35': 'scale',
        'G217_PQ_PN35': 'scale',
        'G217_SQ_PN35': 'scale'
    },
    'Field Values': {
        'G214_PQ_PN35': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G214_SQ_PN35': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G217_PQ_PN35': {
            0.0: 'No',
            1.0: 'Yes',
            7.0: 'Involved in incorrect skip - not answered',
            9.0: 'Not stated'
        },
        'G217_SQ_PN35': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
    }
})

In [ ]:
rprint(m["Field Values"])

{
    'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G217_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in incorrect skip - not answered', 9.0: 'Not stated'},
    'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
}

In [ ]:
PN35 = Metadata(
    label= "Missed work due to pain",
    field_values = {-88: "N/A", -99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

## PN36

In [ ]:
var = "PN36"

### Data

Some of the existing options are the same across all datasets:
- 0: No
- 1: Yes
- 9: Missing

For G214_PQ and G214_SQ
- 8: N/A

For G217_PQ:
- 7: Involved in incorrect skip - not answered

Values of 7 should be considered missing, as discussed with Alex D'Vauz.

Changes:
- Convert 8 to -88
- Convert 9 to -99
- Convert 7 to -99

In [ ]:
unique_vals = unique_values(df, var)
rprint(unique_vals)

{
    'G214_PQ_PN34': (None, 0.0, 1.0, 8.0, 9.0),
    'G214_SQ_PN34': (None, 0.0, 1.0, 8.0, 9.0),
    'G217_PQ_PN34': (None, 0.0, 1.0, 7.0, 9.0),
    'G217_SQ_PN34': (None, 0.0, 1.0, 9.0)
}

In [ ]:
# Note the unique values for PN34 across all datasets combined
value_options = set()
for values in unique_vals.values(): 
    value_options.update(values)
value_options.discard(None)
value_options

{0.0, 1.0, 7.0, 8.0, 9.0}

#### Define properties to test, validate, and explore

- When PN17 is 0 (No), PN34 should be 8 (N/A)
- When PN17 is 1 (Yes), PN34 should be 0, 1, or 9
- When PN17 is 9 (Missing), PN34 should be 9

In [ ]:
test = df.clone().collect()

In [ ]:
for value in (0, 1, 9):
    rprint(f"When PN17 == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}PN17") == value)
            .select(f"{p}{var}")
            .unique()
            .to_dict(as_series=False)
        )

When PN17 == 0

{'G214_PQ_PN34': [8.0]}

{'G214_SQ_PN34': [8.0]}

{'G217_PQ_PN34': [0.0, 7.0, 1.0, 9.0]}

{'G217_SQ_PN34': [0.0, 1.0, 9.0]}

...

When PN17 == 9

{'G214_PQ_PN34': [9.0]}

{'G214_SQ_PN34': [9.0]}

{'G217_PQ_PN34': [9.0]}

{'G217_SQ_PN34': [9.0]}

As per PN25, properties 2 and 3 were as expected and property 1 did not pass due to differences in the Y17 follow-ups.

In [ ]:
for value in value_options:
    rprint(f"When {var} == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}{var}") == value)
            .select(f"{p}PN17")
            .unique()
            .to_dict(as_series=False)
        )

When PN34 == 0.0

{'G214_PQ_PN17': [1.0]}

{'G214_SQ_PN17': [1.0]}

{'G217_PQ_PN17': [0.0, 1.0]}

{'G217_SQ_PN17': [0.0, 1.0]}

...

When PN34 == 9.0

{'G214_PQ_PN17': [9.0]}

{'G214_SQ_PN17': [9.0, 1.0]}

{'G217_PQ_PN17': [9.0, 1.0, 0.0]}

{'G217_SQ_PN17': [9.0, 0.0]}

No additional, unexpected values were found.

### Metadata

Clear discrepancies in labels and field values.

Changes:
- Harmonised label to "Sought professional advice/treatment"
- Updated field values to reflect changes in data

In [ ]:
m = filter_metadata(var, df, meta)

In [ ]:
rprint(m)

defaultdict(<class 'dict'>, {
    'Label': {
        'G214_PQ_PN34': 'Takes meds to relieve back pain',
        'G214_SQ_PN34': 'Takes meds to relieve back pain',
        'G217_PQ_PN34': 'Ever taken medication to relieve back pain',
        'G217_SQ_PN34': 'Take any medication for back pain?'
    },
    'Field Type': {
        'G214_PQ_PN34': 'Numeric',
        'G214_SQ_PN34': 'Numeric',
        'G217_PQ_PN34': 'Numeric',
        'G217_SQ_PN34': 'Numeric'
    },
    'Field Width': {'G214_PQ_PN34': 8, 'G214_SQ_PN34': 8, 'G217_PQ_PN34': 8, 'G217_SQ_PN34': 8},
    'Decimals': {'G214_PQ_PN34': '0', 'G214_SQ_PN34': '0', 'G217_PQ_PN34': '0', 'G217_SQ_PN34': '0'},
    'Variable Type': {
        'G214_PQ_PN34': 'scale',
        'G214_SQ_PN34': 'scale',
        'G217_PQ_PN34': 'scale',
        'G217_SQ_PN34': 'scale'
    },
    'Field Values': {
        'G214_PQ_PN34': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G214_SQ_PN34': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G217_PQ_PN34': {
            0.0: 'No',
            1.0: 'Yes',
            7.0: 'Involved in incorrect skip - not answered',
            9.0: 'Not stated'
        },
        'G217_SQ_PN34': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
    }
})

In [ ]:
rprint(m["Field Values"])

{
    'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G217_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in incorrect skip - not answered', 9.0: 'Not stated'},
    'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
}

In [ ]:
PN34 = Metadata(
    label= "Took medication to relieve pain",
    field_values = {-88: "N/A", -99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

## PN9

In [ ]:
var = "PN9"

### Data

Some of the existing options are the same across all datasets:
- 0: No
- 1: Yes
- 9: Missing

For G214_PQ and G214_SQ
- 8: N/A

For G217_PQ:
- 7: Involved in incorrect skip - not answered

Values of 7 should be considered missing, as discussed with Alex D'Vauz.

Changes:
- Convert 8 to -88
- Convert 9 to -99
- Convert 7 to -99

In [ ]:
unique_vals = unique_values(df, var)
rprint(unique_vals)

{
    'G214_PQ_PN34': (None, 0.0, 1.0, 8.0, 9.0),
    'G214_SQ_PN34': (None, 0.0, 1.0, 8.0, 9.0),
    'G217_PQ_PN34': (None, 0.0, 1.0, 7.0, 9.0),
    'G217_SQ_PN34': (None, 0.0, 1.0, 9.0)
}

In [ ]:
# Note the unique values for PN34 across all datasets combined
value_options = set()
for values in unique_vals.values(): 
    value_options.update(values)
value_options.discard(None)
value_options

{0.0, 1.0, 7.0, 8.0, 9.0}

#### Define properties to test, validate, and explore

- When PN17 is 0 (No), PN34 should be 8 (N/A)
- When PN17 is 1 (Yes), PN34 should be 0, 1, or 9
- When PN17 is 9 (Missing), PN34 should be 9

In [ ]:
test = df.clone().collect()

In [ ]:
for value in (0, 1, 9):
    rprint(f"When PN17 == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}PN17") == value)
            .select(f"{p}{var}")
            .unique()
            .to_dict(as_series=False)
        )

When PN17 == 0

{'G214_PQ_PN34': [8.0]}

{'G214_SQ_PN34': [8.0]}

{'G217_PQ_PN34': [0.0, 7.0, 1.0, 9.0]}

{'G217_SQ_PN34': [0.0, 1.0, 9.0]}

...

When PN17 == 9

{'G214_PQ_PN34': [9.0]}

{'G214_SQ_PN34': [9.0]}

{'G217_PQ_PN34': [9.0]}

{'G217_SQ_PN34': [9.0]}

As per PN25, properties 2 and 3 were as expected and property 1 did not pass due to differences in the Y17 follow-ups.

In [ ]:
for value in value_options:
    rprint(f"When {var} == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}{var}") == value)
            .select(f"{p}PN17")
            .unique()
            .to_dict(as_series=False)
        )

When PN34 == 0.0

{'G214_PQ_PN17': [1.0]}

{'G214_SQ_PN17': [1.0]}

{'G217_PQ_PN17': [0.0, 1.0]}

{'G217_SQ_PN17': [0.0, 1.0]}

...

When PN34 == 9.0

{'G214_PQ_PN17': [9.0]}

{'G214_SQ_PN17': [9.0, 1.0]}

{'G217_PQ_PN17': [9.0, 1.0, 0.0]}

{'G217_SQ_PN17': [9.0, 0.0]}

No additional, unexpected values were found.

### Metadata

Clear discrepancies in labels and field values.

Changes:
- Harmonised label to "Sought professional advice/treatment"
- Updated field values to reflect changes in data

In [ ]:
m = filter_metadata(var, df, meta)

In [ ]:
rprint(m)

defaultdict(<class 'dict'>, {
    'Label': {
        'G214_PQ_PN34': 'Takes meds to relieve back pain',
        'G214_SQ_PN34': 'Takes meds to relieve back pain',
        'G217_PQ_PN34': 'Ever taken medication to relieve back pain',
        'G217_SQ_PN34': 'Take any medication for back pain?'
    },
    'Field Type': {
        'G214_PQ_PN34': 'Numeric',
        'G214_SQ_PN34': 'Numeric',
        'G217_PQ_PN34': 'Numeric',
        'G217_SQ_PN34': 'Numeric'
    },
    'Field Width': {'G214_PQ_PN34': 8, 'G214_SQ_PN34': 8, 'G217_PQ_PN34': 8, 'G217_SQ_PN34': 8},
    'Decimals': {'G214_PQ_PN34': '0', 'G214_SQ_PN34': '0', 'G217_PQ_PN34': '0', 'G217_SQ_PN34': '0'},
    'Variable Type': {
        'G214_PQ_PN34': 'scale',
        'G214_SQ_PN34': 'scale',
        'G217_PQ_PN34': 'scale',
        'G217_SQ_PN34': 'scale'
    },
    'Field Values': {
        'G214_PQ_PN34': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G214_SQ_PN34': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G217_PQ_PN34': {
            0.0: 'No',
            1.0: 'Yes',
            7.0: 'Involved in incorrect skip - not answered',
            9.0: 'Not stated'
        },
        'G217_SQ_PN34': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
    }
})

In [ ]:
rprint(m["Field Values"])

{
    'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G217_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in incorrect skip - not answered', 9.0: 'Not stated'},
    'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
}

In [ ]:
PN34 = Metadata(
    label= "Took medication to relieve pain",
    field_values = {-88: "N/A", -99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

## PN38

In [ ]:
var = "PN38"

### Data

Some of the existing options are the same across all datasets:
- 0: No
- 1: Yes
- 9: Missing

For G214_PQ and G214_SQ
- 8: N/A

For G217_PQ:
- 7: Involved in incorrect skip - not answered

Values of 7 should be considered missing, as discussed with Alex D'Vauz.

Changes:
- Convert 8 to -88
- Convert 9 to -99
- Convert 7 to -99

In [ ]:
unique_vals = unique_values(df, var)
rprint(unique_vals)

{
    'G214_PQ_PN34': (None, 0.0, 1.0, 8.0, 9.0),
    'G214_SQ_PN34': (None, 0.0, 1.0, 8.0, 9.0),
    'G217_PQ_PN34': (None, 0.0, 1.0, 7.0, 9.0),
    'G217_SQ_PN34': (None, 0.0, 1.0, 9.0)
}

In [ ]:
# Note the unique values for PN34 across all datasets combined
value_options = set()
for values in unique_vals.values(): 
    value_options.update(values)
value_options.discard(None)
value_options

{0.0, 1.0, 7.0, 8.0, 9.0}

#### Define properties to test, validate, and explore

- When PN17 is 0 (No), PN34 should be 8 (N/A)
- When PN17 is 1 (Yes), PN34 should be 0, 1, or 9
- When PN17 is 9 (Missing), PN34 should be 9

In [ ]:
test = df.clone().collect()

In [ ]:
for value in (0, 1, 9):
    rprint(f"When PN17 == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}PN17") == value)
            .select(f"{p}{var}")
            .unique()
            .to_dict(as_series=False)
        )

When PN17 == 0

{'G214_PQ_PN34': [8.0]}

{'G214_SQ_PN34': [8.0]}

{'G217_PQ_PN34': [0.0, 7.0, 1.0, 9.0]}

{'G217_SQ_PN34': [0.0, 1.0, 9.0]}

...

When PN17 == 9

{'G214_PQ_PN34': [9.0]}

{'G214_SQ_PN34': [9.0]}

{'G217_PQ_PN34': [9.0]}

{'G217_SQ_PN34': [9.0]}

As per PN25, properties 2 and 3 were as expected and property 1 did not pass due to differences in the Y17 follow-ups.

In [ ]:
for value in value_options:
    rprint(f"When {var} == {value}")
    for p in prefixes:
        rprint(
            test
            .filter(pl.col(f"{p}{var}") == value)
            .select(f"{p}PN17")
            .unique()
            .to_dict(as_series=False)
        )

When PN34 == 0.0

{'G214_PQ_PN17': [1.0]}

{'G214_SQ_PN17': [1.0]}

{'G217_PQ_PN17': [0.0, 1.0]}

{'G217_SQ_PN17': [0.0, 1.0]}

...

When PN34 == 9.0

{'G214_PQ_PN17': [9.0]}

{'G214_SQ_PN17': [9.0, 1.0]}

{'G217_PQ_PN17': [9.0, 1.0, 0.0]}

{'G217_SQ_PN17': [9.0, 0.0]}

No additional, unexpected values were found.

### Metadata

Clear discrepancies in labels and field values.

Changes:
- Harmonised label to "Sought professional advice/treatment"
- Updated field values to reflect changes in data

In [ ]:
m = filter_metadata(var, df, meta)

In [ ]:
rprint(m)

defaultdict(<class 'dict'>, {
    'Label': {
        'G214_PQ_PN34': 'Takes meds to relieve back pain',
        'G214_SQ_PN34': 'Takes meds to relieve back pain',
        'G217_PQ_PN34': 'Ever taken medication to relieve back pain',
        'G217_SQ_PN34': 'Take any medication for back pain?'
    },
    'Field Type': {
        'G214_PQ_PN34': 'Numeric',
        'G214_SQ_PN34': 'Numeric',
        'G217_PQ_PN34': 'Numeric',
        'G217_SQ_PN34': 'Numeric'
    },
    'Field Width': {'G214_PQ_PN34': 8, 'G214_SQ_PN34': 8, 'G217_PQ_PN34': 8, 'G217_SQ_PN34': 8},
    'Decimals': {'G214_PQ_PN34': '0', 'G214_SQ_PN34': '0', 'G217_PQ_PN34': '0', 'G217_SQ_PN34': '0'},
    'Variable Type': {
        'G214_PQ_PN34': 'scale',
        'G214_SQ_PN34': 'scale',
        'G217_PQ_PN34': 'scale',
        'G217_SQ_PN34': 'scale'
    },
    'Field Values': {
        'G214_PQ_PN34': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G214_SQ_PN34': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
        'G217_PQ_PN34': {
            0.0: 'No',
            1.0: 'Yes',
            7.0: 'Involved in incorrect skip - not answered',
            9.0: 'Not stated'
        },
        'G217_SQ_PN34': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
    }
})

In [ ]:
rprint(m["Field Values"])

{
    'G214_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G214_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 8.0: 'Not applicable', 9.0: 'Not stated'},
    'G217_PQ_PN25': {0.0: 'No', 1.0: 'Yes', 7.0: 'Involved in incorrect skip - not answered', 9.0: 'Not stated'},
    'G217_SQ_PN25': {0.0: 'No', 1.0: 'Yes', 9.0: 'Not stated'}
}

In [ ]:
PN34 = Metadata(
    label= "Took medication to relieve pain",
    field_values = {-88: "N/A", -99: "Missing", 0: "No", 1: "Yes"},
    field_type = "Numeric",
    field_width = 3,
    decimals =  0,
    variable_type = "Nominal"
)

## Sandbox

What values exist in subsequent variables when PN17 (and for G217, PN9 and PN38) is 0?

In [ ]:
np.unique(
    df
    .filter(
        pl.col("G217_PQ_PN17") == 0,
        pl.col("G217_PQ_PN9") == 0,
        pl.col("G217_PQ_PN38") == 0
    )
    .select('G217_PQ_PN25', 'G217_PQ_PN34', 'G217_PQ_PN35', 'G217_PQ_PN36', 'G217_PQ_PN38', 'G217_PQ_PN9')
    .collect()
)#.to_pandas().value_counts()

array([0., 9.])

In [ ]:
(
    df
    .filter(
        pl.col("G217_SQ_PN17") == 0,
        pl.col("G217_SQ_PN9") == 0,
        pl.col("G217_SQ_PN38") == 0
    )
    .select("G217_SQ_PN17", 'G217_SQ_PN9', 'G217_SQ_PN38', 'G217_SQ_PN25', 'G217_SQ_PN34', 'G217_SQ_PN35', 'G217_SQ_PN36')
    # .unique()
)

<LazyFrame at 0x2604F727620>

In [ ]:
(
    df
    .filter(
        pl.col("G217_PQ_PN17") == 0,
    )
    .select("G217_PQ_PN17", 'G217_PQ_PN9', 'G217_PQ_PN38', 'G217_PQ_PN25', 'G217_PQ_PN34', 'G217_PQ_PN35', 'G217_PQ_PN36')
    # .unique()
)

<LazyFrame at 0x2604F724230>